In [5]:
# Split test - val dataset
import os
import json
import shutil
import random

# Define paths
data_dir = "./archive"

test_mask_path = os.path.join(data_dir, "test/masks")
test_img_path = os.path.join(data_dir, "test/img")
test_ann_path = os.path.join(data_dir, "test/ann")

val_mask_path = os.path.join(data_dir, "val/masks")
val_img_path = os.path.join(data_dir, "val/img")
val_ann_path = os.path.join(data_dir, "val/ann")

# Get a list of all annotation files
test_img_files = os.listdir(test_img_path)
random.shuffle(test_img_files)

# Split dataset
split_idx = int(0.8 * len(test_img_files))
val_img_files = test_img_files[:split_idx]
test_img_files = test_img_files[split_idx:]

# Move annotations and corresponding images to val/
for img_file in val_img_files:
    img_src = os.path.join(test_img_path, img_file)
    img_dest = os.path.join(val_img_path, img_file)
    shutil.move(img_src, img_dest)

    # Move corresponding images
    mask_filename = img_file.replace(".jpg", ".png")
    ann_filename = img_file.replace(".jpg", ".jpg.json")
    mask_src = os.path.join(test_mask_path, mask_filename)
    mask_dest = os.path.join(val_mask_path, mask_filename)
    ann_src = os.path.join(test_ann_path, ann_filename)
    ann_dest = os.path.join(val_ann_path, ann_filename)
    shutil.move(mask_src, mask_dest)
    shutil.move(ann_src, ann_dest)
print("Test dataset successfully split into validation and test")

Test dataset successfully split into validation and test


In [6]:
# Overlapping mask and image for debugging
!pip install opencv-python -q
import cv2
import numpy as np
import os

# Paths
data_dir = "./archive"
val_mask_path = os.path.join(data_dir, "val/masks")
val_img_path = os.path.join(data_dir, "val/img")
output_path = "./archive/val/overlayed" 

os.makedirs(output_path, exist_ok=True)  # Create folder if it doesn't exist

# List all images
for img_file in os.listdir(val_img_path):
    if img_file.endswith(".jpg"):
        img_path = os.path.join(val_img_path, img_file)
        
        # Construct corresponding mask filename
        mask_filename = os.path.splitext(img_file)[0] + ".png"
        mask_path = os.path.join(val_mask_path, mask_filename)

        if os.path.exists(mask_path):
            # Load the image and mask
            image = cv2.imread(img_path)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

            # Convert grayscale mask to color for visualization
            color_mask = cv2.applyColorMap(mask, cv2.COLORMAP_JET)  # Apply a color map

            # Blend image and mask using transparency (alpha blending)
            alpha = 0.5  # Transparency level (adjust as needed)
            overlay = cv2.addWeighted(image, 1, color_mask, alpha, 0)

            # Save and display
            overlay_path = os.path.join(output_path, mask_filename)
            cv2.imwrite(overlay_path, overlay)
            print(f"Saved overlay image: {overlay_path}")

            # Display (optional)
            # cv2.imshow("Overlay", overlay)
            # cv2.waitKey(0)  # Press any key to continue
            # cv2.destroyAllWindows()

Saved overlay image: ./archive/val/overlayed\00000048.png
Saved overlay image: ./archive/val/overlayed\00004401.png
Saved overlay image: ./archive/val/overlayed\00004402.png
Saved overlay image: ./archive/val/overlayed\00004403.png
Saved overlay image: ./archive/val/overlayed\00004404.png
Saved overlay image: ./archive/val/overlayed\00004405.png
Saved overlay image: ./archive/val/overlayed\00004408.png
Saved overlay image: ./archive/val/overlayed\00004409.png
Saved overlay image: ./archive/val/overlayed\00004410.png
Saved overlay image: ./archive/val/overlayed\00004411.png
Saved overlay image: ./archive/val/overlayed\00004412.png
Saved overlay image: ./archive/val/overlayed\00004413.png
Saved overlay image: ./archive/val/overlayed\00004414.png
Saved overlay image: ./archive/val/overlayed\00004415.png
Saved overlay image: ./archive/val/overlayed\00004416.png
Saved overlay image: ./archive/val/overlayed\00004417.png
Saved overlay image: ./archive/val/overlayed\00004418.png
Saved overlay 

In [10]:
# Resize mask and images
from PIL import Image
import torch
from torchvision import transforms

def resize_image_and_mask(image, mask, size = (512, 512)):
    resize_transform = transforms.Resize(size)

    # Resize the image
    image_resized = resize_transform(image)

    # Resize the mask (must be done with nearest neighbor to preserve category labels)
    mask_resized = resize_transform(mask)

    return image_resized, mask_resized

# Dir paths
"""
# for test images
img_dir = "./archive/train/img"
mask_dir = "./archive/train/masks"

output_img_dir = "./archive/train/resized_train_img"
output_mask_dir = "./archive/train/resized_train_masks"
"""

# for val images
img_dir = "./archive/val/img"
mask_dir = "./archive/val/masks"

output_img_dir = "./archive/val/resized_val_img"
output_mask_dir = "./archive/val/resized_val_masks"

# Create the output directories if they don't exist
os.makedirs(output_img_dir, exist_ok = True)
os.makedirs(output_mask_dir, exist_ok = True)

# Get the list of image filenames in the training directory
image_files = os.listdir(img_dir)

# Loop through all images and their corresponding masks
for img_file in image_files:
    # Load the image and mask
    img_path = os.path.join(img_dir, img_file)
    mask_path = os.path.join(mask_dir, img_file.replace('.jpg', '.png'))

    # Open the image and mask using PIL
    img = Image.open(img_path).convert('RGB')
    mask = Image.open(mask_path).convert('L')

    # Resize the image and mask
    resized_img, resized_mask = resize_image_and_mask(img, mask, size = (512, 512))

    # Save the resized image and mask
    resized_img.save(os.path.join(output_img_dir, img_file))
    resized_mask.save(os.path.join(output_mask_dir, img_file.replace('.jpg', '_mask.png')))